# Predicting Spectral Library from Fasta

In [20]:
%reload_ext autoreload
%autoreload 2

Predict fasta libray and save as HDF file using this notebook.
And then use [alphapeptdeep_hdf_to_tsv.ipynb](alphapeptdeep_hdf_to_tsv.ipynb) to translate hdf into tsv (diann/spectronaut) format.

#### Prepare the data and settings

In [21]:
from alphabase.peptide.fragment import get_charged_frag_types
import pandas as pd

fasta_list = [
   "uniprotkb_proteome_UP000005640_AND_prot_2025_03_10.fasta"
]

# output spectral library in hdf format
hdf_path = "uniprotkb_proteome_UP000005640_AND_prot_2025_03_10.hdf"

protease="trypsin"
nce = 30
instrument = 'timsTOF'

add_phos=False

protease_dict = {
    "trypsin": "([KR])", # this is in fact the "trypsin/P"
    "lysc": "([K])",
    "lysn": "\w(?=K)",
}
min_pep_len = 7
max_pep_len = 35
max_miss_cleave = 1
max_var_mods = 1
min_pep_mz = 400
max_pep_mz = 1200
precursor_charge_min = 2
precursor_charge_max = 4

var_mods = []
var_mods += ['Oxidation@M']
#var_mods += ['Phospho@S','Phospho@T','Phospho@Y']


frag_types = get_charged_frag_types(
    ['b','y']+
    (['b_modloss','y_modloss'] if add_phos else []), 
    2
)

In [22]:
digest = protease_dict[protease] # Or digest = "trypsin/P"

`protease` and `digest` are designed by regular expression. alphabase provides several built-in enzymes, we don't need to design the regular expression for most of the enzymes. Here are all the built-in enzymes:

In [23]:
from alphabase.protein.fasta import protease_dict
protease_dict

{'arg-c': 'R',
 'asp-n': '\\w(?=D)',
 'bnps-skatole': 'W',
 'caspase 1': '(?<=[FWYL]\\w[HAT])D(?=[^PEDQKR])',
 'caspase 2': '(?<=DVA)D(?=[^PEDQKR])',
 'caspase 3': '(?<=DMQ)D(?=[^PEDQKR])',
 'caspase 4': '(?<=LEV)D(?=[^PEDQKR])',
 'caspase 5': '(?<=[LW]EH)D',
 'caspase 6': '(?<=VE[HI])D(?=[^PEDQKR])',
 'caspase 7': '(?<=DEV)D(?=[^PEDQKR])',
 'caspase 8': '(?<=[IL]ET)D(?=[^PEDQKR])',
 'caspase 9': '(?<=LEH)D',
 'caspase 10': '(?<=IEA)D',
 'chymotrypsin high specificity': '([FY](?=[^P]))|(W(?=[^MP]))',
 'chymotrypsin low specificity': '([FLY](?=[^P]))|(W(?=[^MP]))|(M(?=[^PY]))|(H(?=[^DMPW]))',
 'chymotrypsin': '([FLY](?=[^P]))|(W(?=[^MP]))|(M(?=[^PY]))|(H(?=[^DMPW]))',
 'clostripain': 'R',
 'cnbr': 'M',
 'enterokinase': '(?<=[DE]{3})K',
 'factor xa': '(?<=[AFGILTVM][DE]G)R',
 'formic acid': 'D',
 'glutamyl endopeptidase': 'E',
 'glu-c': 'E',
 'granzyme b': '(?<=IEP)D',
 'hydroxylamine': 'N(?=G)',
 'iodosobenzoic acid': 'W',
 'lys-c': 'K',
 'lys-n': '\\w(?=K)',
 'ntcb': '\\w(?=C)',
 'peps

#### Initialize a `PredictSpecLibFasta` object

In [24]:
from peptdeep.protein.fasta import PredictSpecLibFasta
from peptdeep.pretrained_models import ModelManager

model_mgr = ModelManager(device='gpu')

model_mgr.nce = nce
model_mgr.instrument = instrument

fasta_lib = PredictSpecLibFasta(
    model_mgr, 
    protease=digest,
    charged_frag_types=frag_types, 
    var_mods=var_mods, 
    fix_mods=['Carbamidomethyl@C'],
    max_missed_cleavages=max_miss_cleave,
    max_var_mod_num=max_var_mods,
    peptide_length_max=max_pep_len,
    peptide_length_min=min_pep_len,
    precursor_charge_min=precursor_charge_min,
    precursor_charge_max=precursor_charge_max,
    precursor_mz_min=min_pep_mz,
    precursor_mz_max=max_pep_mz,
    decoy=None
)

/home/anshan/miniconda3/envs/env_faiss_gpu/lib/python3.8/site-packages/peptdeep/model/model_interface.py:731: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(stream

#### Digest

In [25]:
fasta_lib.get_peptides_from_fasta_list(fasta_list)

If we have a sequence DataFrame (`seq_df`) containing peptide sequences in the `sequence` column, we can skip `get_peptides_from_fasta_list`. Just assign `seq_df` to `fasta_lib._precursor_df` and perform all following steps.

```
fasta_lib._precursor_df = seq_df
```

#### Append decoy sequences and add modifications

In [26]:
fasta_lib.append_decoy_sequence()
fasta_lib.add_modifications()

We will get a protein DataFrame (`protein_df`) after digestion

In [27]:
fasta_lib.protein_df

,protein_id,full_name,gene_name,gene_org,description,sequence
0,A0A087X0I6,tr|A0A087X0I6|A0A087X0I6_HUMAN,MARK1,A0A087X0I6_HUMAN,tr|A0A087X0I6|A0A087X0I6_HUMAN non-specific se...,MSARTPLPTVNERDTENHTSVDGYTEPHIQPTKSSSRQNIPRCRNS...
1,A0A0A0MRJ0,tr|A0A0A0MRJ0|A0A0A0MRJ0_HUMAN,CDC42BPA,A0A0A0MRJ0_HUMAN,tr|A0A0A0MRJ0|A0A0A0MRJ0_HUMAN non-specific se...,MSGEVRLRQLEQFILDGPAQTNGQCFSVETLLDILICLYDECNNSP...
2,A0A0A0MRJ1,tr|A0A0A0MRJ1|A0A0A0MRJ1_HUMAN,CDC42BPA,A0A0A0MRJ1_HUMAN,tr|A0A0A0MRJ1|A0A0A0MRJ1_HUMAN non-specific se...,MSGEVRLRQLEQFILDGPAQTNGQCFSVETLLDILICLYDECNNSP...
3,A0A0B4J1S7,tr|A0A0B4J1S7|A0A0B4J1S7_HUMAN,PTPN22,A0A0B4J1S7_HUMAN,tr|A0A0B4J1S7|A0A0B4J1S7_HUMAN protein-tyrosin...,MDQREILQKFLDEAQSKKITKEEFANEFLKLKRQSTKYKADKTYPT...
4,A0A1B0GTB8,tr|A0A1B0GTB8|A0A1B0GTB8_HUMAN,CPT2,A0A1B0GTB8_HUMAN,tr|A0A1B0GTB8|A0A1B0GTB8_HUMAN Carnitine O-pal...,MVPRLLLRAWPRGPAVGPGAPSRPLSAGSGPGQYLQRSIVPTMHYQ...
...,...,...,...,...,...,...
6940,X6RK96,tr|X6RK96|X6RK96_HUMAN,TRMT1L,X6RK96_HUMAN,tr|X6RK96|X6RK96_HUMAN tRNA methyltransferase ...,TSVNYLDSAFRNIRNLGIVSVTSTDISSLYAKAQHVARRHYGCNIV...
6941,X6RKN2,tr|X6RKN2|X6RKN2_HUMAN,NFASC,X6RKN2_HUMAN,tr|X6RKN2|X6RKN2_HUMAN Neurofascin (Fragment) ...,IEIPMDLTQPPTITKQSAKDHIVDPRDNILIECEAKGNPAPSFHWT...
6942,X6RKU3,tr|X6RKU3|X6RKU3_HUMAN,FBXO42,X6RKU3_HUMAN,tr|X6RKU3|X6RKU3_HUMAN F-box protein 42 (Fragm...,MSSPPQIVIDDATILILGGCGGPNALFKDAWLLHMHSGPWAWQPLK...
6943,X6RL62,tr|X6RL62|X6RL62_HUMAN,OMA1,X6RL62_HUMAN,tr|X6RL62|X6RL62_HUMAN OMA1 zinc metallopeptid...,MSFICGLQSAARNHVFFRFNSLSNWRKCNTLASTSRGCHQVQVNHI...


`precursor_df` contains the main information of peptides.

In [28]:
fasta_lib.precursor_df['nAA'] = fasta_lib.precursor_df.sequence.str.len()
fasta_lib.precursor_df.sort_values('nAA', inplace=True)
fasta_lib.precursor_df.reset_index(drop=True, inplace=True)

In [29]:
fasta_lib.precursor_df

,sequence,protein_idxes,miss_cleavage,is_prot_nterm,is_prot_cterm,mods,mod_sites,nAA
0,QWLNKFK,1043;3159,1,False,False,,,7
1,MVANCCR,1075;2411;2423;2434;2760;2761;2762;2764;4333,0,False,False,Oxidation@M;Carbamidomethyl@C;Carbamidomethyl@C,1;5;6,7
2,MVANCCR,1075;2411;2423;2434;2760;2761;2762;2764;4333,0,False,False,Carbamidomethyl@C;Carbamidomethyl@C,5;6,7
3,SLSDVAR,989,0,False,False,,,7
4,GCSDSLR,989,0,False,False,Carbamidomethyl@C,2,7
...,...,...,...,...,...,...,...,...
213966,SALGECLAAFAGAFPVAFLETHLDKHNIYSIYNTK,1075;2411;2423;2434;2760;2761;2762;2764,1,False,False,Carbamidomethyl@C,6,35
213967,AELGSPLYNNEPFAIMLFGMVTKFCSGHAPHFPMK,811,1,False,False,Oxidation@M;Carbamidomethyl@C,16;25,35
213968,AELGSPLYNNEPFAIMLFGMVTKFCSGHAPHFPMK,811,1,False,False,Oxidation@M;Carbamidomethyl@C,20;25,35
213969,PGAHEIEVGVDEEGFIYLAFEAPEAPDSSEFQWSK,819,0,False,False,,,35


Check `precursor_df` after adding charge states.

In [30]:
fasta_lib.add_charge()
fasta_lib.precursor_df

test_add_charge_1


,sequence,protein_idxes,miss_cleavage,is_prot_nterm,is_prot_cterm,mods,mod_sites,nAA,charge
0,QWLNKFK,1043;3159,1,False,False,,,7,2
1,QWLNKFK,1043;3159,1,False,False,,,7,3
2,QWLNKFK,1043;3159,1,False,False,,,7,4
3,MVANCCR,1075;2411;2423;2434;2760;2761;2762;2764;4333,0,False,False,Oxidation@M;Carbamidomethyl@C;Carbamidomethyl@C,1;5;6,7,2
4,MVANCCR,1075;2411;2423;2434;2760;2761;2762;2764;4333,0,False,False,Oxidation@M;Carbamidomethyl@C;Carbamidomethyl@C,1;5;6,7,3
...,...,...,...,...,...,...,...,...,...
641908,PGAHEIEVGVDEEGFIYLAFEAPEAPDSSEFQWSK,819,0,False,False,,,35,3
641909,PGAHEIEVGVDEEGFIYLAFEAPEAPDSSEFQWSK,819,0,False,False,,,35,4
641910,SPVQCLPPASSGCAPSSGGCGPSSEGGCFLNHHRR,1249,1,False,False,Carbamidomethyl@C;Carbamidomethyl@C;Carbamidom...,5;13;20;28,35,2
641911,SPVQCLPPASSGCAPSSGGCGPSSEGGCFLNHHRR,1249,1,False,False,Carbamidomethyl@C;Carbamidomethyl@C;Carbamidom...,5;13;20;28,35,3


`PredictSpecLibFasta.calc_precursor_mz` will append a `precursor_mz` column to the `precursor_df`.

`PredictSpecLibFasta.hash_precursor_df` will append `mod_seq_hash` and `mod_seq_charge_hash` columns to the `precursor_df`. `mod_seq_hash` column contains the unique signatures (np.int64) for corresponding peptides ( `sequence`,`mods` and `mod_sites`). `mod_seq_charge_hash` column contains the unique signatures (np.int64) for corresponding precursors ( `sequence`,`mods`, `mod_sites` and `charge`). 

In [31]:
fasta_lib.hash_precursor_df()
fasta_lib.calc_precursor_mz()
fasta_lib.precursor_df

,sequence,protein_idxes,miss_cleavage,is_prot_nterm,is_prot_cterm,mods,mod_sites,nAA,charge,mod_seq_hash,mod_seq_charge_hash,precursor_mz
0,QWLNKFK,1043;3159,1,False,False,,,7,2,10573461572310657859,10573461572310657861,482.274170
1,QWLNKFK,1043;3159,1,False,False,,,7,3,10573461572310657859,10573461572310657862,321.851872
2,QWLNKFK,1043;3159,1,False,False,,,7,4,10573461572310657859,10573461572310657863,241.640723
3,MVANCCR,1075;2411;2423;2434;2760;2761;2762;2764;4333,0,False,False,Oxidation@M;Carbamidomethyl@C;Carbamidomethyl@C,1;5;6,7,2,6983161399497124296,6983161399497124298,463.685690
4,MVANCCR,1075;2411;2423;2434;2760;2761;2762;2764;4333,0,False,False,Oxidation@M;Carbamidomethyl@C;Carbamidomethyl@C,1;5;6,7,3,6983161399497124296,6983161399497124299,309.459552
...,...,...,...,...,...,...,...,...,...,...,...,...
641908,PGAHEIEVGVDEEGFIYLAFEAPEAPDSSEFQWSK,819,0,False,False,,,35,3,4285894207192834970,4285894207192834973,1294.267003
641909,PGAHEIEVGVDEEGFIYLAFEAPEAPDSSEFQWSK,819,0,False,False,,,35,4,4285894207192834970,4285894207192834974,970.952072
641910,SPVQCLPPASSGCAPSSGGCGPSSEGGCFLNHHRR,1249,1,False,False,Carbamidomethyl@C;Carbamidomethyl@C;Carbamidom...,5;13;20;28,35,2,5347820895146747718,5347820895146747720,1834.303920
641911,SPVQCLPPASSGCAPSSGGCGPSSEGGCFLNHHRR,1249,1,False,False,Carbamidomethyl@C;Carbamidomethyl@C;Carbamidom...,5;13;20;28,35,3,5347820895146747718,5347820895146747721,1223.205039


#### Predict MS2/RT/CCS(mobility)

In [32]:
fasta_lib.precursor_df['instrument'] = model_mgr.instrument
fasta_lib.precursor_df['nce'] = model_mgr.nce
res = fasta_lib.model_manager.predict_all(
    fasta_lib.precursor_df,
    predict_items=['rt','mobility','ms2'],
    frag_types = frag_types,
)
fasta_lib.set_precursor_and_fragment(
    **res
)

2025-03-12 16:35:23> Predicting RT ...


  0%|          | 0/29 [00:00<?, ?it/s]

100%|██████████| 29/29 [01:01<00:00,  2.12s/it]

2025-03-12 16:36:24> Predicting mobility ...



100%|██████████| 29/29 [01:05<00:00,  2.26s/it]


2025-03-12 16:37:38> Predicting MS2 ...


100%|██████████| 29/29 [05:22<00:00, 11.11s/it]


Check memory usage for the library

In [33]:
import os, psutil
import numpy as np
process = psutil.Process(os.getpid())
print(f'{len(fasta_lib.precursor_df)*1e-6:.2f}M precursors with {np.prod(fasta_lib.fragment_mz_df.values.shape, dtype=float)*(1e-6):.2f}M fragments used {process.memory_info().rss/1024**3:.4f} GB memory')

0.64M precursors with 40.24M fragments used 2.2239 GB memory


The predicted fragment intensities

In [34]:
fasta_lib.fragment_intensity_df

,b_z1,b_z2,y_z1,y_z2
0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.200403,0.0
2,0.0,0.0,1.000000,0.0
3,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.000000,0.0
...,...,...,...,...
10060678,0.0,0.0,0.000000,0.0
10060679,0.0,0.0,0.000000,0.0
10060680,0.0,0.0,0.000000,0.0
10060681,0.0,0.0,0.000000,0.0


In [42]:
fasta_lib.fragment_intensity_df[0:6+1]

,b_z1,b_z2,y_z1,y_z2
0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.200403,0.0
2,0.0,0.0,1.000000,0.0
3,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.000000,0.0
5,0.0,0.0,0.000000,0.0
6,0.0,0.0,0.000000,0.0


In [35]:
print(fasta_lib.fragment_intensity_df["b_z1"].unique())
len(fasta_lib.fragment_intensity_df["b_z1"].unique())

[0.         0.06036784 0.04568237 ... 0.06247457 0.13296014 0.05249897]


2569209

The calculated fragment m/z values

In [36]:
fasta_lib.fragment_mz_df

,b_z1,b_z2,y_z1,y_z2
0,129.065857,65.036568,835.482483,418.244873
1,315.145172,158.076218,649.403198,325.205231
2,428.229218,214.618256,536.319092,268.663177
3,542.272156,271.639709,422.276184,211.641724
4,670.367126,335.687195,294.181213,147.594254
...,...,...,...,...
10060678,2949.227051,1475.117188,719.380798,360.194031
10060679,3063.270020,1532.138672,605.337891,303.172577
10060680,3200.328857,1600.668091,468.278961,234.643127
10060681,3337.387695,1669.197510,331.220062,166.113663


In [41]:
fasta_lib.fragment_mz_df[0:6+1]

,b_z1,b_z2,y_z1,y_z2
0,129.065857,65.036568,835.482483,418.244873
1,315.145172,158.076218,649.403198,325.205231
2,428.229218,214.618256,536.319092,268.663177
3,542.272156,271.639709,422.276184,211.641724
4,670.367126,335.687195,294.181213,147.594254
5,817.435547,409.221405,147.112808,74.060043
6,129.065857,65.036568,835.482483,418.244873


`PredictSpecLibFasta.rt_to_irt_pred` will translate the predicted RT values to iRT values (`rt_pred` to `irt_pred`). This is useful for DiaNN and Spectronaut search.

In [37]:
fasta_lib.translate_rt_to_irt_pred()
fasta_lib.precursor_df

Predict RT for 11 iRT precursors.
Linear regression of `rt_pred` to `irt`:
   R_square         R       slope  intercept  test_num
0   0.99007  0.995022  152.235622  -39.23216        11


,sequence,protein_idxes,miss_cleavage,is_prot_nterm,is_prot_cterm,mods,mod_sites,nAA,charge,mod_seq_hash,...,precursor_mz,instrument,nce,rt_pred,rt_norm_pred,ccs_pred,mobility_pred,frag_start_idx,frag_stop_idx,irt_pred
0,QWLNKFK,1043;3159,1,False,False,,,7,2,10573461572310657859,...,482.274170,timsTOF,30,0.376854,0.376854,340.025116,0.836941,0,6,18.138489
1,QWLNKFK,1043;3159,1,False,False,,,7,3,10573461572310657859,...,321.851872,timsTOF,30,0.376854,0.376854,431.047089,0.707333,6,12,18.138489
2,QWLNKFK,1043;3159,1,False,False,,,7,4,10573461572310657859,...,241.640723,timsTOF,30,0.376854,0.376854,489.396790,0.602321,12,18,18.138489
3,MVANCCR,1075;2411;2423;2434;2760;2761;2762;2764;4333,0,False,False,Oxidation@M;Carbamidomethyl@C;Carbamidomethyl@C,1;5;6,7,2,6983161399497124296,...,463.685690,timsTOF,30,0.000000,0.000000,314.762024,0.774321,18,24,-39.232160
4,MVANCCR,1075;2411;2423;2434;2760;2761;2762;2764;4333,0,False,False,Oxidation@M;Carbamidomethyl@C;Carbamidomethyl@C,1;5;6,7,3,6983161399497124296,...,309.459552,timsTOF,30,0.000000,0.000000,358.338318,0.587689,24,30,-39.232160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641908,PGAHEIEVGVDEEGFIYLAFEAPEAPDSSEFQWSK,819,0,False,False,,,35,3,4285894207192834970,...,1294.267003,timsTOF,30,0.871227,0.871227,696.765930,1.155668,10060513,10060547,93.399592
641909,PGAHEIEVGVDEEGFIYLAFEAPEAPDSSEFQWSK,819,0,False,False,,,35,4,4285894207192834970,...,970.952072,timsTOF,30,0.871227,0.871227,831.044373,1.033790,10060547,10060581,93.399592
641910,SPVQCLPPASSGCAPSSGGCGPSSEGGCFLNHHRR,1249,1,False,False,Carbamidomethyl@C;Carbamidomethyl@C;Carbamidom...,5;13;20;28,35,2,5347820895146747718,...,1834.303920,timsTOF,30,0.417084,0.417084,575.817993,1.432294,10060581,10060615,24.262861
641911,SPVQCLPPASSGCAPSSGGCGPSSEGGCFLNHHRR,1249,1,False,False,Carbamidomethyl@C;Carbamidomethyl@C;Carbamidom...,5;13;20;28,35,3,5347820895146747718,...,1223.205039,timsTOF,30,0.417084,0.417084,634.046021,1.051421,10060615,10060649,24.262861


In [40]:
fasta_lib.precursor_df.columns

Index(['sequence', 'protein_idxes', 'miss_cleavage', 'is_prot_nterm',
       'is_prot_cterm', 'mods', 'mod_sites', 'nAA', 'charge', 'mod_seq_hash',
       'mod_seq_charge_hash', 'precursor_mz', 'instrument', 'nce', 'rt_pred',
       'rt_norm_pred', 'ccs_pred', 'mobility_pred', 'frag_start_idx',
       'frag_stop_idx', 'irt_pred'],
      dtype='object')

#### Save the library into a HDF5 (.hdf) file

In [38]:
fasta_lib.save_hdf(hdf_path)
hdf_path

'uniprotkb_proteome_UP000005640_AND_prot_2025_03_10.hdf'

Now use [alphapeptdeep_hdf_to_tsv.ipynb](alphapeptdeep_hdf_to_tsv.ipynb) to translate hdf into TSV (diann/spectronaut) format. Translation is quite slow because writing TSV file is slow for large libraries.